In [6]:
!nvidia-smi

Unable to determine the device handle for GPU 0000:18:00.0: Unknown Error


In [4]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
 

In [5]:
from hi_sam.text_segmentation import load_sam_predictor, make_text_segmentation_args


arg = make_text_segmentation_args(
    model_type='vit_l',
    checkpoint_path='sam_tss_l_textseg.pth',
    input_size=(512, 512),
    hier_det=False,
)

sam = load_sam_predictor(arg)


/home/s2/mlvu25/anaconda3/envs/tf/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Freeze image encoder.


sam_vit_l_0b3195.pth:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

<All keys matched successfully>


RuntimeError: No CUDA GPUs are available

In [ ]:
from diffusers.utils import load_image
from hi_sam.text_segmentation import run_text_stroke_segmentation


hf_dataset_base_url = "https://huggingface.co/datasets/GoGiants1/TMDBEval500/resolve/main/TMDBEval500/images/"
input_image = load_image(hf_dataset_base_url + '1.jpg')

mask_w_patchify = run_text_stroke_segmentation(sam_detector=sam, input_path=input_image, patch_mode=True)

In [ ]:
display(mask_w_patchify)

In [ ]:
mask_wo_patchify = run_text_stroke_segmentation(sam_detector=sam, input_path=input_image, patch_mode=False)

In [ ]:
display(mask_wo_patchify)